In [7]:
import re
% load_ext autotime
import sqlalchemy as sal
import pandas as pd

time: 7.49 s (started: 2021-09-25 10:28:42 +07:00)


In [23]:
engine = sal.create_engine('sqlite:////Users/quoccuongnguyen/Documents/TDTU/YEAR-4/HK1-2021-2022/Massive-Data/Code/Database-SQLLite/exam.db', echo=True)
conn = engine.connect()

time: 4.79 ms (started: 2021-09-25 10:42:20 +07:00)


In [24]:
metadata = sal.MetaData()
users = sal.Table('users', metadata,
                  sal.Column('id', sal.Integer, primary_key=True),
                  sal.Column('name', sal.String),
                  sal.Column('fullname', sal.String))

addresses = sal.Table('addresses', metadata,
                      sal.Column('id', sal.Integer, primary_key=True),
                      sal.Column('user_id', None, sal.ForeignKey('users.id')),
                      sal.Column('email_address', sal.String, nullable=False))

metadata.create_all(engine)

2021-09-25 10:42:23,697 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-25 10:42:23,699 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2021-09-25 10:42:23,700 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-25 10:42:23,702 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2021-09-25 10:42:23,703 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-25 10:42:23,704 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2021-09-25 10:42:23,706 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-25 10:42:23,707 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("addresses")
2021-09-25 10:42:23,708 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-25 10:42:23,711 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2021-09-25 10:42:23,713 INFO sqlalchemy.engine.Engine [no key 0.00126s] ()
2021-09-25 10:42:23,716 INFO sqlalchemy.engine.Engine 
CREATE TABLE addresses

In [25]:
ins = users.insert().values(name='jack', fullname='Jack Jones')
print(str(ins))
print(ins.compile().params)

INSERT INTO users (name, fullname) VALUES (:name, :fullname)
{'name': 'jack', 'fullname': 'Jack Jones'}
time: 3.57 ms (started: 2021-09-25 10:42:27 +07:00)


In [26]:
result = conn.execute(ins)
print(result)

2021-09-25 10:42:30,059 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2021-09-25 10:42:30,061 INFO sqlalchemy.engine.Engine [generated in 0.00233s] ('jack', 'Jack Jones')
2021-09-25 10:42:30,064 INFO sqlalchemy.engine.Engine COMMIT
time: 8.63 ms (started: 2021-09-25 10:42:30 +07:00)


In [27]:
ins.bind = engine
print(str(ins))

INSERT INTO users (name, fullname) VALUES (?, ?)
time: 1.18 ms (started: 2021-09-25 10:42:33 +07:00)


In [28]:
print(result.inserted_primary_key)

(1,)
time: 1.44 ms (started: 2021-09-25 10:42:33 +07:00)


In [29]:
conn.execute(users.insert(), id=2, name='wendy', fullname='Wendy Williams')

conn.execute(addresses.insert(), [
    {'user_id': 1, 'email_address': 'jack@yahoo.com'},
    {'user_id': 1, 'email_address': 'jack@msn.com'},
    {'user_id': 2, 'email_address': 'www@www.org'},
    {'user_id': 2, 'email_address': 'wendy@aol.com'},
])

2021-09-25 10:45:17,412 INFO sqlalchemy.engine.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2021-09-25 10:45:17,413 INFO sqlalchemy.engine.Engine [generated in 0.00167s] (2, 'wendy', 'Wendy Williams')
2021-09-25 10:45:17,415 INFO sqlalchemy.engine.Engine COMMIT
2021-09-25 10:45:17,418 INFO sqlalchemy.engine.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2021-09-25 10:45:17,419 INFO sqlalchemy.engine.Engine [generated in 0.00108s] ((1, 'jack@yahoo.com'), (1, 'jack@msn.com'), (2, 'www@www.org'), (2, 'wendy@aol.com'))
2021-09-25 10:45:17,420 INFO sqlalchemy.engine.Engine COMMIT


time: 16.4 ms (started: 2021-09-25 10:45:17 +07:00)


In [19]:
# Select * from user
df = pd.DataFrame(conn.execute(sal.select([users])))
df

2021-09-25 10:35:00,202 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname 
FROM users
2021-09-25 10:35:00,205 INFO sqlalchemy.engine.Engine [cached since 48.93s ago] ()


,0,1,2
0,1,jack,Jack Jones
1,2,wendy,Wendy Williams


time: 14.7 ms (started: 2021-09-25 10:35:00 +07:00)


In [30]:
# fetchone -> return 1 line data
# fetchone similar to pop()

# get 1st line from users
row = conn.execute(sal.select([users])).fetchone()
print("name:", row['name'], "; fullname:", row['fullname'])

# get next line from users
row = conn.execute(sal.select([users])).fetchone()
print("name:", row[1], "; fullname:", row[2])

# get all lines from users
for row in conn.execute(sal.select([users])):
    print("name:", row[users.c.name], "; fullname:", row[users.c.fullname])

2021-09-25 10:47:30,959 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname 
FROM users
2021-09-25 10:47:30,960 INFO sqlalchemy.engine.Engine [generated in 0.00174s] ()
name: jack ; fullname: Jack Jones
2021-09-25 10:47:30,964 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname 
FROM users
2021-09-25 10:47:30,965 INFO sqlalchemy.engine.Engine [cached since 0.005985s ago] ()
name: jack ; fullname: Jack Jones
2021-09-25 10:47:30,967 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname 
FROM users
2021-09-25 10:47:30,967 INFO sqlalchemy.engine.Engine [cached since 0.008732s ago] ()
name: jack ; fullname: Jack Jones
name: wendy ; fullname: Wendy Williams
time: 11.5 ms (started: 2021-09-25 10:47:30 +07:00)


In [34]:
# users.c.name -> table users column name
result = conn.execute(sal.select([users.c.name, users.c.fullname]))

for row in result:
    print(row)

2021-09-25 10:50:41,824 INFO sqlalchemy.engine.Engine SELECT users.name, users.fullname 
FROM users
2021-09-25 10:50:41,827 INFO sqlalchemy.engine.Engine [cached since 22.13s ago] ()
('jack', 'Jack Jones')
('wendy', 'Wendy Williams')
time: 6.61 ms (started: 2021-09-25 10:50:41 +07:00)


In [37]:
if users.c.id == addresses.c.user_id:
    print('ok')
else:
    print('no')

no
time: 980 µs (started: 2021-09-25 10:55:48 +07:00)
